In [1]:
import pandas as pd
import numpy as np
from numpy import load
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from scipy.linalg import norm
from PIL import Image
from numpy import asarray
import os

In [2]:
### import data from csv (written in data_processing)
q_array = load('q_array0.npy')
a_array = load('a_array0.npy')
N = 100

### find number of trials
data_trials = int(a_array.shape[0])


In [3]:

q_array = np.resize(q_array,((N,16,100)))
a_array = np.resize(a_array,((N,6)))
q_array_end = np.zeros((N,16))
for i in range(N):
    q_array_end[i,:] = q_array[i,:,-1]
print(q_array_end.shape)
print(a_array.shape)

(100, 16)
(100, 6)


In [4]:

## Open the image form working directory

xy_array = np.zeros((N,288,432,4))
xz_array = np.zeros((N,288,432,4))

for i in range(N):
    
    img1 = Image.open('/home/bowen/Documents/Rod_manipulation/Flexible-Tool/png_data/'+str(i)+'_xy.png')
    img2 = Image.open('/home/bowen/Documents/Rod_manipulation/Flexible-Tool/png_data/'+str(i)+'_xz.png')

    xy_array[i,:,:,:] = asarray(img1)
    xz_array[i,:,:,:] = asarray(img2)


In [5]:
## load the image data into numpy arries and transfer RGBA channels into one channel by taking mean of RGB data.
## May have function directly address this channel transfer (check later)
for k in range(N):
    xy_array[k,:,:,0] = np.mean(xy_array[k,:,:,0] + xy_array[k,:,:,1] + xy_array[k,:,:,2])
    xz_array[k,:,:,0] = np.mean(xz_array[k,:,:,0] + xz_array[k,:,:,1] + xz_array[k,:,:,2])

xy_array = np.resize(xy_array,(N,288,432,1))
xz_array = np.resize(xz_array,(N,288,432,1))
print(xy_array.shape)
print(xz_array.shape)

## put two image arraies into one single array with two channels and then reshape it because of the format of pytorch
image_array = np.stack([xy_array,xz_array], axis = -1).squeeze()
image_array = np.transpose(image_array,[0,3,1,2]) ## change the axis
print(image_array.shape)


(100, 288, 432, 1)
(100, 288, 432, 1)
(100, 2, 288, 432)


In [6]:
# with np.printoptions(threshold=np.inf):
    
#     print(xy_array(0,288,432,1))

In [7]:
## split q_end(end configurations) and a_array into test and train set and put them into tensor and dataloader


qend_train_x = (q_array_end[0:int(0.8*N),:]).astype(np.float32)

qend_test_x = (q_array_end[int(0.8*N):N,:]).astype(np.float32) 
qend_test_x = np.expand_dims(qend_test_x, 1) # add dimension for neural net
print(qend_test_x.shape)

train_y = (a_array[0:int(0.8*N),:]).astype(np.float32)
test_y = (a_array[int(0.8*N):N,:]).astype(np.float32)

qx_test_tensor = torch.from_numpy(qend_test_x)
y_test_tensor = torch.from_numpy(test_y)
test_data_qtoa = [(qx_test_tensor[i],y_test_tensor[i]) for i in range(qx_test_tensor.shape[0])]

qx_train_tensor = torch.from_numpy(qend_train_x)
y_train_tensor = torch.from_numpy(train_y)

print(qx_train_tensor.size(0))
print(y_train_tensor.size(0))


# q_train_dataset = torch.utils.data.TensorDataset(qx_train_tensor, y_train_tensor)
# q_test_dataset = torch.utils.data.TensorDataset(qx_test_tensor, y_test_tensor)

# q_train_loader = torch.utils.data.DataLoader(q_train_dataset, batch_size = 5, shuffle=True)
# q_test_loader = torch.utils.data.DataLoader(q_test_dataset, batch_size = 5, shuffle=True)

# for batch, (x, y) in enumerate(q_train_loader):
#     print("batch", batch)
#     print("Shape of x: ", x.shape, x.dtype)
#     print("Shape of y: ", y.shape, y.dtype)
#     break

(20, 1, 16)
80
80


In [8]:
## split q_end(end configurations) and a_array into test and train set and put them into tensor and dataloader
image_train_x = (image_array[0:int(0.8*N),:,:,:]).astype(np.float32) 
image_test_x = (image_array[int(0.8*N):N,:,:,:]).astype(np.float32) 
image_test_x = np.expand_dims(image_test_x, 1) # add dimension for neural net


imagex_test_tensor = torch.from_numpy(image_test_x)

test_data = [(imagex_test_tensor[i],y_test_tensor[i]) for i in range(imagex_test_tensor.shape[0])]

imagex_train_tensor = torch.from_numpy(image_train_x)



# image_train_dataset = torch.utils.data.TensorDataset(imagex_train_tensor, y_train_tensor)
# image_test_dataset = torch.utils.data.TensorDataset(imagex_test_tensor, y_test_tensor)

# image_train_loader = torch.utils.data.DataLoader(image_train_dataset, batch_size = 5, shuffle=True)
# image_test_loader = torch.utils.data.DataLoader(image_test_dataset, batch_size = 5, shuffle=True)

# for batch, (x, y) in enumerate(image_train_loader):
#     print("batch", batch)
#     print("Shape of x: ", x.shape, x.dtype)
#     print("Shape of y: ", y.shape, y.dtype)
#     break
    
    
train_dataset = torch.utils.data.TensorDataset(imagex_train_tensor, qx_train_tensor, y_train_tensor)
test_dataset = torch.utils.data.TensorDataset(imagex_test_tensor, qx_test_tensor, y_test_tensor)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 5, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 5, shuffle=True)

for batch, (img,q, y) in enumerate(train_loader):
    print("batch", batch)
    print("Shape of img: ", img.shape, img.dtype)
    print("Shape of q: ", q.shape, q.dtype)
    print("Shape of y: ", y.shape, y.dtype)
    break


batch 0
Shape of img:  torch.Size([5, 2, 288, 432]) torch.float32
Shape of q:  torch.Size([5, 16]) torch.float32
Shape of y:  torch.Size([5, 6]) torch.float32


In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.pool = nn.MaxPool2d(2)
        self.relu = nn.functional.relu

        self.conv1 = nn.Conv2d(in_channels=2, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv5 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)

        ## the input features should be the flatten version of the outchannel from the last conv2d layer
        self.linear1 = nn.Linear(9872, 128)
        self.linear2 = nn.Linear(128, 32)
        self.linear3 = nn.Linear(32, 6)

    def forward(self, img, q):
#         print('in')
#         img = self.conv1(img)
#         print('out')
#         img_out = self.relu(img)
#         print('out2')
#         img_out = self.pool(img_out)
#         print('out3')
        img_out = self.pool(self.relu(self.conv1(img)))
        img_out = self.pool(self.relu(self.conv2(img_out)))
        img_out = self.pool(self.relu(self.conv3(img_out)))
        img_out = self.pool(self.relu(self.conv4(img_out)))
        img_out = self.pool(self.relu(self.conv5(img_out)))

        img_out = nn.Flatten()(img_out)
        print(img_out.size())

        concat = torch.cat([img_out, q], 1)
        print(concat.size())
        print('1')
        output = self.relu(self.linear1(concat))
        print('2')
        output = self.relu(self.linear2(output))  ## should be 5*128 x 128*32 = 5*32
        print('3')
        output = nn.Sigmoid()(self.linear3(output))
        print('4')

        return output
model = Net()
print(model)

Net(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(2, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (linear1): Linear(in_features=9872, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=32, bias=True)
  (linear3): Linear(in_features=32, out_features=6, bias=True)
)


In [10]:
### loss function and optimizer for training 
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
       

In [11]:
### Functions for training
def train(dataloader, model, loss_fn, optimizer,loss_list):
    size = len(dataloader.dataset)
    for batch, (img,q,y) in enumerate(dataloader):

#         print("batch ", batch)
#         print("Shape of x: ", x.shape)
#         print("Shape of y: ", y.shape)
    
        # Compute prediction error
        pred = model(img, q)
#         print("pred", pred)
#         print("y", y)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(x)
            loss_list.append(loss)
        if batch % 20 == 0:
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss_list

In [12]:
### function for testing the model
def test(dataloader, model,test_loss_list):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for img,q, y in dataloader:
            pred = model(img,q)
            test_loss += loss_fn(pred, y)
    print(f"Test Error: Loss = {test_loss:>8f} \n")
    test_loss_list.append(test_loss)
    return test_loss_list

In [13]:
### Train the model
epochs = 50
loss_list=[]
test_loss_list=[]
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    loss_list = train(train_loader, model, loss_fn, optimizer,loss_list)
    test_loss_list = test(test_loader, model, test_loss_list)

Epoch 1
-------------------------------
torch.Size([5, 9856])
torch.Size([5, 9872])
1
2
3
4


NameError: name 'x' is not defined